In [45]:
# Packages
import socket
import pickle
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [46]:
# Load the dataframes dictionary from the file
with open("../dataframes/duration_dataframes_dict.pkl", "rb") as file:
    dataframes = pickle.load(file)

city = "Liverpool"
df = dataframes[city]
print(df)

           city  duration  year  nduration ensemble
0     Liverpool         1  1980   103001.0       01
1     Liverpool         1  1982   104201.0       01
2     Liverpool         1  1984   105559.0       01
3     Liverpool         1  1986   109290.0       01
4     Liverpool         1  1988   113000.0       01
...         ...       ...   ...        ...      ...
5962  Liverpool         9  2072     7159.0     mean
5963  Liverpool         9  2074     7095.0     mean
5964  Liverpool         9  2076     6906.0     mean
5965  Liverpool         9  2078     6959.0     mean
5966  Liverpool         9  2080     7025.0     mean

[5967 rows x 5 columns]


In [47]:
# Function to find a free port
def get_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))  # Let the OS choose a free port
        return s.getsockname()[1]

In [ ]:
# Create Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("City Heatwave Durations by Ensemble"),

    html.Div([
        html.Label("Select City:"),
        dcc.Dropdown(
            id="city-dropdown",
            options=[{"label": city.replace("_", " "), "value": city} for city in dataframes],
            value=list(dataframes.keys())[0]
        )
    ], style={"width": "48%", "display": "inline-block"}),

    html.Div([
        html.Label("Select Duration:"),
        dcc.Dropdown(
            id="duration-dropdown",
            options=[
                {"label": f"{i} day" if i == 1 else f"{i} days", "value": f"{i} day" if i == 1 else f"{i} days"}
                for i in range(1, 10)
            ],
            value="1 day"
        )
    ], style={"width": "48%", "display": "inline-block", "float": "right"}),

    dcc.Graph(id="city-plot"),

    #SECOND FIGURE

    html.Hr(),  # Horizontal line to separate figures

    html.H2("Heatwave Durations Over Time by City and Ensemble"),

    html.Div([
        html.Label("Select City:"),
        dcc.Dropdown(
            id="city2-dropdown",
            options=[{"label": city.replace("_", " "), "value": city} for city in dataframes],
            value=list(dataframes.keys())[0]
        )
    ], style={"width": "48%", "display": "inline-block"}),

    html.Div([
        html.Label("Select Ensemble:"),
        dcc.Dropdown(
            id="ensemble-dropdown",
            options=[],  # Will populate dynamically based on city selection
            value=None
        )
    ], style={"width": "48%", "display": "inline-block", "float": "right"}),

    dcc.Graph(id="duration-plot")

])

In [49]:
@app.callback(
    Output("city-plot", "figure"),
    Input("city-dropdown", "value"),
    Input("duration-dropdown", "value")
)

def update_city_plot(selected_city, selected_duration):
    df = dataframes[selected_city]
    duration_int = int(selected_duration.split()[0])
    filtered_df = df[df["duration"] == duration_int]
    fig = px.line(
        filtered_df,
        x="year",
        y="nduration",
        color="ensemble",
        title=f"{duration_int} day Heatwave Duration for {selected_city.replace('_', ' ')}",
        markers=True
    )

    for trace in fig.data:
        if trace.name == "mean":
            trace.line.color = "black"
            trace.line.dash = "dash"

    fig.update_layout(yaxis_title="nduration", xaxis_title="year")
    return fig

In [50]:
@app.callback(
    Output("ensemble-dropdown", "options"),
    Output("ensemble-dropdown", "value"),
    Input("city2-dropdown", "value")
)
def update_ensemble_options(selected_city):
    df = dataframes[selected_city]
    ensembles = sorted(df["ensemble"].unique())
    options = [{"label": ens, "value": ens} for ens in ensembles]
    return options, ensembles[0]  # default to first ensemble

In [51]:
@app.callback(
    Output("duration-plot", "figure"),
    Input("city2-dropdown", "value"),
    Input("ensemble-dropdown", "value")
)
def update_duration_plot(selected_city, selected_ensemble):
    df = dataframes[selected_city]
    
    # Filter by ensemble
    filtered_df = df[df["ensemble"] == selected_ensemble]

    # Plot with color by duration
    fig = px.line(
        filtered_df,
        x="year",
        y="nduration",
        color="duration",
        title=f"nduration for {selected_city.replace('_', ' ')} ensemble: {selected_ensemble}",
        markers=True
    )
    
    fig.update_layout(yaxis_title="nduration", xaxis_title="year")
    return fig

if __name__ == "__main__":
    port = get_free_port()
    print(f"App running on http://127.0.0.1:{port}")
    app.run(debug=True, port=port)


App running on http://127.0.0.1:49995
